# Rabi cycles in the GdW$_{30}$ model system with and without dissipation

In [ ]:
try:
    get_ipython
    isnotebook = True
except:
    isnotebook = False

import os
import sys
import numpy as np
import scipy as sp
import matplotlib
if not isnotebook:
    matplotlib.use('Agg')
import matplotlib.pyplot as plt
from qutip import *

In [ ]:
import qocttools
import qocttools.models.GdW30 as GdW30
import qocttools.hamiltonians as hamiltonians
import qocttools.math_extra as math_extra
import qocttools.pulses as pulses
import qocttools.qoct as qoct
import qocttools.solvers as solvers

In [ ]:
qocttools.about()

In [ ]:
data = []

# Introduction

The purpose of this notebook is to replicate the Rabi-cycle experiments performed on the GdW$_{30}$ system reported in [Jenkins et al., F. Luis, Phys. Rev. B 95, 064423 (2017)]. We show propagations in the presence of a sinusoidal magnetic (microwave) perturbation, with and without dissipation terms, and observe the Rabi cycles.

# Model

The model is defined by the Hamiltonian:

\begin{equation}
        \hat{H}(t) = \hat{H}_0 + f(t)\hat{V}
\end{equation}
where the time-independent part is given by:
\begin{equation}
        \hat{H}_0 = D\bigg[\hat{S}_z^2 - \frac{1}{3}S(S + 1)\bigg] + E[\hat{S}_x^2 - \hat{S}_y^2] - g\mu_B\hat{\vec{S}}\cdot\vec{H}
\end{equation}
and the time-dependent part is:
\begin{equation}
        \hat{H}(t) = \hat{H}_0 + f(t)\hat{V}
\end{equation}
The perturbation is a magnetic field:
\begin{equation}
        \hat{V} = -g\mu_B\hat{\vec{S}}\cdot\vec{H}_m 
\end{equation}

In this case:

* $S = 7/2$

* $D$ = 1281 MHz

* $E$ = 294 MHz

* $\vec{H} = (0.0, 0.0, 0.0615)$ T

* $\vec{H}_m = (0.001, 0.0, 0.0)$ T

In [ ]:
S = 7/2 # spin
E = 294 # value in MHz
D = 1281 # value in MHz
dim = int(2*S + 1) #matrix dim
A = 0.163 #perturbation amplitude in mT

In [ ]:
H = np.array([0.0, 0, 0.615], dtype = float) #magnetic field in T
H_m = np.array([0.001, 0, 0], dtype = float) #only in presence of perturbation (T)

H0 = GdW30.hGdW30(D, E, H)
V = GdW30.vGdW30(H_m)

Sx = jmat(S, "x")
Sy = jmat(S, "y")
Sz = jmat(S, "z")
Sp = np.sqrt(0.2)*(Sx + (1j*Sy))
Sm = np.sqrt(0.2)*(Sx - (1j*Sy))

In [ ]:
# The code will operate in the basis of eigenstates of H0
eigenvalues, eigenstates = H0.eigenstates()
H_new = H0.transform(eigenstates) - eigenvalues[0]
V_new = V.transform(eigenstates)

In [ ]:
w = np.zeros(dim-1)
for i in range(dim - 1):
    w[i] = eigenvalues[i+1] - eigenvalues[i] #in MHz

In [ ]:
tau = 2.0 * np.pi/w[0]
Omega = np.abs(V_new.data[0,1])* 2.0 * A
ncycles = 6
T = ncycles * np.pi/ Omega

# Definition of the pulse

In [ ]:
u = np.array([A, w[0]])
def fu(t, u):
    return 2*u[0]*np.cos(u[1]*t + np.pi)

f = pulses.pulse('user_defined', T, u)
f.assign_user_defined_function(fu, None)
f.set_parameters(u)

# Time array definition

In [ ]:
time = math_extra.timegrid(H0, T, 100)
print("The total propagation time is", T)
print("Delta t = ", (time[1]-time[0])/(2.0*np.pi))
print("The period tau of the fist excitation is", tau/(2.0*np.pi))
print("tau / (Delta t) ) = ", tau / (time[1]-time[0]))
print("The number of time steps is", time.size)

# Reading experimental data file

We have the results from the experiment in this file. It provides <Sy>(t) as a function of time (in ns).

In [ ]:
filename = 'GdW30_Toffoli_Syvstime_15dB.dat'
exp_data = np.loadtxt(filename, delimiter="\t")

In [ ]:
t_exp = exp_data[:, 0]/1000.0
data_exp = exp_data[:, 1]

# Propagation without dissipation terms

The initial state is simply the ground state.

In [ ]:
psi0 = basis(dim, 0)

We propagate using the qutip propagator.

In [ ]:
result = solvers.solve('sesolve', hamiltonians.hamiltonian(H_new, V_new), f, psi0, time)

In the experiments, measurements are made in a rotating frame, that rotates around the $z$ axis, with the cavity frequency that is tuned to the first excitation energy. Therefore, the observable that we look at (Sy) must
be transformed to this rotating frame:

\begin{equation}
S_y^r(t) = e^{i\omega_0 t S_z} S_y e^{-i\omega_0 t S_z}
\end{equation}

We will compute both $S_y$ and $S_z$ (this last one does not need to be transformed).

In [ ]:
syval = np.zeros(time.size)
szval = np.zeros(time.size)
for i in range(time.size):
    #szval[i] = expect(Sx, result.states[i])
    Syt = (1j*time[i]*w[0]*Sz).expm() * Sy * (-1j*time[i]*w[0]*Sz).expm()
    syval[i] = expect( Syt, result.states[i] )
    szval[i] = expect( Sz, result.states[i] ) - S
    #szval[i] = np.vdot(result[i], np.matmul(Sz.full(), result[i]))

In [ ]:
fig, ax = plt.subplots()
ax.set_xlabel('time ($\mu$s)')
ax.set_ylabel('<$S_z(t)$>')
ax.set_xlim(0, T/(2.0*np.pi))
ax.grid()
ax.plot(time/(2*np.pi), syval, label = '$S_y$ (simulation)')
ax.plot(time/(2*np.pi), szval, label = '$S_z - 7/2$ (simulation)')
ax.plot(t_exp, data_exp, label = 'exp. (15dB)')
ax.legend(loc = 'upper center')

fig.savefig('Sz.pdf', format = 'pdf')
if isnotebook:
    plt.show()

I still do not understand this plot. The frequencies of $S_y$ and $S_z$ seem to be different.

# Propagation with dissipation term

As the system is interacting with its enviroment we will use the Lindblad master equation:

\begin{equation}
    \dot{\rho}(t) = -\frac{i}{\hbar}[H(t), \rho(t)] + \sum_n\left(C_n\rho(t)C_n^\dagger - \frac{1}{2}\{C_nC_n^\dagger, \rho(t)\}\right)
\end{equation}

Being $C_n = \sqrt{\gamma_n}A_n$ the collapse operators, $A_n$  the operators through which the environment couples to the system and $\gamma_n$ are the corresponding rates.

We take $n=1$, so:
* $C_1 = S^+ = S_x + iS_y$

* $\gamma_1 = \sqrt{0.2}$

In [ ]:
result = solvers.solve('sesolve', hamiltonians.hamiltonian(H_new, V_new, [Sp]), f, psi0, time)

In [ ]:
syval = np.zeros(time.size)
szval = np.zeros(time.size)
for i in range(time.size):
    szval[i] = expect(Sz, result.states[i]) - S
    Syt = (1j*time[i]*w[0]*Sz).expm() * Sy * (-1j*time[i]*w[0]*Sz).expm()
    syval[i] = expect( Syt, result.states[i] )

data.append(syval[-1])
data.append(szval[-1])

In [ ]:
fig, ax = plt.subplots()
ax.set_xlabel('time ($\mu$s)')
ax.set_ylabel('<$S_z(t)$>')
ax.set_xlim(0, T/(2.0*np.pi))
ax.grid()
ax.plot(time/(2*np.pi), syval, label = '$S_y$ (simulation)')
ax.plot(time/(2*np.pi), szval, label = '$S_z - 7/2$ (simulation)')
ax.plot(t_exp, data_exp, label = 'exp. (15dB)')
ax.legend(loc = 'upper center')

fig.savefig('Sy-diss.pdf', format = 'pdf')
if isnotebook:
    plt.show()

In [ ]:
with open("data", "w") as datafile:
    for i in data:
        datafile.write("{:.14e}\n".format(i))